# ML Graph Embedding methods

In [1]:
#import libraries
import torch
import time
from torch_geometric.nn import Node2Vec

In [2]:
print(torch.version.cuda)

11.8


In [3]:
# Load the graph from the file
server_graph_data_filepath = './data/server_graph_data.pth'
graph_data = torch.load(server_graph_data_filepath)

C:\Users\PC\AppData\Local\Temp\ipykernel_1192\4280630462.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  graph_data = torch.load(server_graph_data_filepath)


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
print(device)
graph_data = graph_data.to(device)

cuda


In [5]:
# Initialize the Node2Vec model
node2vec = Node2Vec(
    graph_data.edge_index,       # Edge list
    embedding_dim=3,      # Size of embeddings
    walk_length=20,        # Length of each random walk
    context_size=10,       # Window size for Skip-Gram
    walks_per_node=10,     # Number of walks per node
    num_negative_samples=1,  # Number of negative samples for Skip-Gram
    p=0.25,  # Return parameter: encourages staying close to the starting node
    q=4.0,   # In-out parameter: encourages exploring further away
    sparse=True            # Use sparse gradients for efficiency
).to(device)

# Define the optimizer
optimizer = torch.optim.SparseAdam(list(node2vec.parameters()), lr=0.01)

# Training loop
def train():
    node2vec.train()
    total_loss = 0
    loader = node2vec.loader(batch_size=8, shuffle=True)
    for pos_rw, neg_rw in loader:
        pos_rw = pos_rw.to(device)
        neg_rw = neg_rw.to(device)
        
        optimizer.zero_grad()
        loss = node2vec.loss(pos_rw, neg_rw)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Run training for multiple epochs
for epoch in range(1, 101):
    # Save timestamp
    start = time.time()
    loss = train()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    # Save timestamp
    end = time.time()
    
    print(end - start)

# Obtain the node embeddings
node_embeddings = node2vec.embedding.weight.data

print("Node Embeddings Shape:", node_embeddings.shape)


KeyboardInterrupt: 

In [6]:
node_embeddings

tensor([[-0.1627,  0.0256,  0.1298],
        [-0.2320, -0.0875,  0.1060],
        [-0.2535,  0.3937,  0.0748],
        ...,
        [-0.1484, -0.1693, -0.0323],
        [ 1.7283, -0.0232, -0.8917],
        [-0.1192, -0.2461,  0.0336]], device='cuda:0')

In [7]:
node2vector_embedding_filepath = 'node2vector_embeddings.pt'
torch.save(node_embeddings, node2vector_embedding_filepath)

In [10]:
# Initialize the DeepWalk model by setting p and q to 1 (unbiased random walks)
deepwalk = Node2Vec(
    graph_data.edge_index,
    embedding_dim=64,
    walk_length=40, # Longer walk length for DeepWalk
    context_size=10,
    walks_per_node=10,
    sparse=True
).to(device)

# Define the optimizer
optimizer = torch.optim.SparseAdam(list(deepwalk.parameters()), lr=0.01)

# Training loop
def train():
    deepwalk.train()
    total_loss = 0
    loader = deepwalk.loader(batch_size=8, shuffle=True)
    for pos_rw, neg_rw in loader:
        pos_rw = pos_rw.to(device)
        neg_rw = neg_rw.to(device)
        
        optimizer.zero_grad()
        loss = deepwalk.loss(pos_rw, neg_rw)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Run training for multiple epochs
for epoch in range(1, 101):
    start = time.time()
    loss = train()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        
    # Save timestamp
    end = time.time()
    
    print(end - start)

# Obtain the node embeddings
node_embeddings_deepwalk = deepwalk.embedding.weight.data

print("Node Embeddings Shape:", node_embeddings_deepwalk.shape)

5.88263463973999
6.081462383270264
5.550729990005493
5.454274892807007
5.481947898864746
5.508334398269653
5.706878423690796
5.7682945728302
5.742300510406494
Epoch: 010, Loss: 0.7363
5.662375211715698
5.6383819580078125
5.702561855316162
5.886704206466675
5.679514169692993
5.608670711517334
5.606393098831177
5.613248348236084
5.710547924041748
5.755554676055908
Epoch: 020, Loss: 0.7299
5.705645561218262
5.62106728553772
5.7871668338775635
5.575872182846069
5.560431957244873
5.538564443588257
5.612644910812378
5.65003228187561
5.710684061050415
5.590488433837891


KeyboardInterrupt: 

In [9]:
node_embeddings_deepwalk

tensor([[-1.8289e-01,  2.0963e-01,  9.8698e-02,  ...,  1.5152e-02,
         -1.9687e-01,  6.6291e-02],
        [ 4.3452e-02, -2.3687e-01, -4.3718e-02,  ..., -6.0400e-02,
         -7.0542e-02,  3.7939e-02],
        [ 2.9276e-03, -1.3018e-01,  9.8565e-02,  ..., -4.3489e-02,
         -2.1407e-01,  2.0897e-01],
        ...,
        [-4.0324e-02,  1.0460e-01,  4.2056e-02,  ...,  1.1005e-01,
          6.1901e-02,  1.4247e-04],
        [ 4.0746e-01, -4.9000e-01,  7.5014e-01,  ...,  2.6305e-01,
         -1.2246e-02,  2.0184e-01],
        [ 1.7585e-01, -1.7938e-01,  1.2310e-01,  ...,  1.9934e-01,
         -4.7857e-03, -9.7514e-02]], device='cuda:0')

In [10]:
deepwalk_embedding_filepath = 'deepwalk_embeddings.pt'
torch.save(node_embeddings_deepwalk, deepwalk_embedding_filepath)